In [1]:
import mne
import mne_bids
from mne_bids import BIDSPath
import os, os.path as op
import nilearn
import matplotlib.pyplot as plt
import numpy as np
import nih2mne

#from nilearn import *
from nih2mne.utilities.bids_helpers import get_mri_dict
n_jobs = 10 #Number of parrallel operations

In [ ]:
# first let's set up directories

#bids_root = '/Users/nugenta/meg_workshop_data'
bids_root = op.join('/data',os.environ['USER'], 'meg_workshop_data') 
deriv_root = op.join(bids_root, 'derivatives')
project_root = op.join(deriv_root, 'Day1')
output_dir = op.join(deriv_root,'Day2')
fs_subjects_dir = op.join(deriv_root,'freesurfer','subjects')

In [ ]:
# pick a subject - same one from time frequency

subject = 'ON03748'
fs_subject = 'sub-' + subject
data_dict = nih2mne.utilities.bids_helpers.get_mri_dict(subject,bids_root, task='airpuff')

In [ ]:
# load in all the MRI stuff

bem = data_dict['bem'].load()
fwd = data_dict['fwd'].load()
src = fwd['src']
trans = data_dict['trans'].load()

In [ ]:
#Load the Raw data
bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
# put a bandpass on the raw, and a notch
raw.filter(0.3, 50, n_jobs=n_jobs)
raw.notch_filter(freqs=[60])
# also make some filtered data in individual bands for later
theta=raw.copy().filter(4,8)
alpha=raw.copy().filter(8,12)
gamma=raw.copy().filter(30,50)

In [ ]:
# We also want to load in the empty room dataset so we can calculate a noise covariance 
bids_path = BIDSPath(root=bids_root, subject=subject, task='noise', run='01',session='01')
noise = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
nosie_alpha = noise.copy().filter(8,12)

In [ ]:
# Now lets extract the events, make epochs
evts, evtsid = mne.events_from_annotations(raw)
epochs = mne.Epochs(raw, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)

# make the filtered epochs as well
epochs_alpha = mne.Epochs(alpha, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)
epochs_theta = mne.Epochs(theta, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)
epochs_gamma = mne.Epochs(gamma, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)

# let's also average the epochs to visualize the evoked responses
evk_stim = epochs['stim'].average()
evk_missingstim = epochs['missingstim'].average()

In [ ]:
%matplotlib inline
_=evk_stim.plot()
_=evk_missingstim.plot()

In [ ]:
cov_all = mne.compute_covariance(epochs, tmin=-0.1, tmax = 0.2, n_jobs=n_jobs)

In [ ]:
# Sanity check the covariance
c = cov_all.data
cinv = np.linalg.inv(c)
print(np.trace(cinv)/c.shape[0])

# Plot eigenvalue spectrum -- have to sort them first
w, v = np.linalg.eig(c)
w = list(w)
w.sort(key = lambda x: -x)

plt.plot(np.log(w))

In [ ]:
noise_cov = mne.make_ad_hoc_cov(raw.info)

In [ ]:
# Let's say we want to project the evoked response into source space
# notice here that I'm using the covariance from ALL the marks, not just the stimuli
filters = mne.beamformer.make_lcmv(evk_stim.info, fwd, cov_all, reg=0.05, pick_ori='max-power',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')

In [ ]:
stc=mne.beamformer.apply_lcmv(evk_stim, filters)

In [ ]:
brain=stc.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# You probably noticed the dreaded Beamformer Sign Ambiguity 

In [ ]:
# what happens if we use the normals from the freesurfer cortical surface? 
fwd_src_ori = mne.convert_forward_solution(fwd, surf_ori=True)
filters_src_ori = mne.beamformer.make_lcmv(evk_stim.info, fwd_src_ori, cov_all, reg=0.05, pick_ori='normal',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')
stc_src_ori=mne.beamformer.apply_lcmv(evk_stim, filters)

In [ ]:
brain=stc_src_ori.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# You might *think* that looks worse, but now, the sign of the output is following the surface. 
# In opposing sulci, the surface normals are oriented opposite eachother. 

In [ ]:
# Frequently what we do is to invoke a sign "flip"

In [ ]:
# I've figured out that index 166 corresponds to roughly the peak of the evoked response.
# There are 8196 vertices
for i in range(8196):
    if stc.data[i,166] < 0:
        stc.data[i,:] *= -1
# You need to remember here, however, that you are also flipping vertices that aren't particularly active
# so you'll also be ensuring that all the noise in that time point is positive.

In [ ]:
brain=stc.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# We can make the time course for the missing stim as well
stc_missing=mne.beamformer.apply_lcmv(evk_missingstim, filters)

In [ ]:
# plot it 
brain=stc_missing.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# interesting, there does seem to be something out around 150ms, doesn't there.... but again the flip thing...

In [ ]:
# Remember our time frequency plots for this person - wasn't there something in alpha around that time? 
# Maybe we should look at alpha power

In [ ]:
# We'll make covariance matrices for all the events, then stim and missingstim
cov_all_alpha = mne.compute_covariance(epochs_alpha, tmin=0.1, tmax = 0.2, n_jobs=n_jobs)
cov_stim_alpha = mne.compute_covariance(epochs_alpha['stim'], tmin=.1, tmax = 0.2, n_jobs=n_jobs)
cov_missingstim_alpha = mne.compute_covariance(epochs_alpha['missingstim'], tmin=0.1, tmax = 0.2, n_jobs=n_jobs)

In [ ]:
# calculate the beamformer using all the events
filters_alpha = mne.beamformer.make_lcmv(epochs_alpha.info, fwd, cov_all_alpha, reg=0.05, pick_ori='max-power',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')

In [ ]:
# Now get the source estimates for stim and missingstim
stc_stim_alpha = mne.beamformer.apply_lcmv_cov(cov_stim_alpha, filters_alpha)
stc_missing_alpha = mne.beamformer.apply_lcmv_cov(cov_missingstim_alpha, filters_alpha)

In [ ]:
# We can look at the ratio here, or the log10 ratio if you like
stc_contrast=stc_stim_alpha.copy()
stc_contrast.data=np.log10(stc_stim_alpha.data/stc_missing_alpha.data)

In [ ]:
brain=stc_contrast.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
brain=stc_stim_alpha.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
brain=stcs_missing_stim[1].plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# Okay, now lets loop over all the subjects!!
subjects=['ON02811','ON03748','ON22671','ON42107','ON52662','ON61373','ON62003','ON70467','ON72082',
         'ON84896','ON85305','ON89474','ON89475']
fs_subject = 'sub-' + subject
data_dict = nih2mne.utilities.bids_helpers.get_mri_dict(subject,bids_root, task='airpuff')
fs_subjects=[]
for subject in subjects:
    fs_subjects.append('sub-' + subject)

In [ ]:
# declare a few empty lists to hold the source estimates

stcs_stim=[]
stcs_missing_stim=[]
stcs_contrast=[]

for subject in subjects:
    # filenames and load MRI files
    fs_subject = 'sub-' + subject
    data_dict = nih2mne.utilities.bids_helpers.get_mri_dict(subject,bids_root, task='airpuff')
    bem = data_dict['bem'].load()
    fwd = data_dict['fwd'].load()
    src = fwd['src']
    trans = data_dict['trans'].load()
    #Load the Raw data
    bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
    raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
    # filter for alpha        
    alpha=raw.copy().filter(8,12)
    evts, evtsid = mne.events_from_annotations(raw)
    epochs_alpha = mne.Epochs(alpha, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)
    cov_all_alpha = mne.compute_covariance(epochs_alpha, tmin=0.1, tmax = 0.2, n_jobs=n_jobs)
    cov_stim_alpha = mne.compute_covariance(epochs_alpha['stim'], tmin=.1, tmax = 0.2, n_jobs=n_jobs)
    cov_missingstim_alpha = mne.compute_covariance(epochs_alpha['missingstim'], tmin=0.1, tmax = 0.2, n_jobs=n_jobs)
    filters_alpha = mne.beamformer.make_lcmv(epochs_alpha.info, fwd, cov_all_alpha, reg=0.05, pick_ori='max-power',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')
    stc_stim_alpha = mne.beamformer.apply_lcmv_cov(cov_stim_alpha, filters_alpha)
    stc_missing_alpha = mne.beamformer.apply_lcmv_cov(cov_missingstim_alpha, filters_alpha)
    stc_contrast=stc_stim_alpha.copy()
    stc_contrast.data=stc_stim_alpha.data/stc_missing_alpha.data
    stcs_stim.append(stc_stim_alpha)
    stcs_missing_stim.append(stc_missing_alpha)
    stcs_contrast.append(stc_contrast)

In [ ]:
# you can look at any one of these
subjectsnum=1
brain=stcs_contrast[subjectsnum].plot(hemi='both', subjects_dir=fs_subjects_dir, 
                                      subject=fs_subjects[subjectsnum])

In [ ]:
# we are going to want to morph these all to standard space - in this case fsaverage
fname_fsaverage_src = op.join(fs_subjects_dir,'fsaverage','bem','fsaverage-ico-5-src.fif')
src_to = mne.read_source_spaces(fname_fsaverage_src)

In [ ]:
# make a new list to hold our morphed source estimates
stcs_contrast_morphed = []
for i in range(len(subjects)):
    # compute the morph for a given subject
    morph = mne.compute_source_morph(stcs_contrast[i], subject_from=fs_subjects[i], subject_to='fsaverage',
                                src_to=src_to, subjects_dir=fs_subjects_dir)
    # apply the morph
    stcs_contrast_morphed.append(morph.apply(stcs_contrast[i]))

In [ ]:
# just for visualization, lets make an average source estimate
src_avg_data = np.zeros(np.shape(stcs_contrast_morphed[0].data))
for i in range(len(subjects)):
    src_avg_data += stcs_contrast_morphed[i].data
src_avg=stcs_contrast_morphed[0].copy()
src_avg.data=(src_avg_data/len(subjects))

In [ ]:
# look at that estimate
brain = src_avg.plot(hemi='both', subjects_dir=fs_subjects_dir, subject='fsaverage')

In [ ]:
# Ah! That doesn't look at all like that one subject we looked at initially. Be careful of making 
# generalizations based on a single individual. 

In [ ]:
# What about gamma - - I seem to remember that there is a gamma response to 
# many sensory stimuli, including somatosensory.

In [ ]:
# Let's see what happens if we look at power changes in gamma over time. 

In [ ]:
subject = 'ON02811'
fs_subject = 'sub-' + subject
data_dict = nih2mne.utilities.bids_helpers.get_mri_dict(subject,bids_root, task='airpuff')
bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
# load in all the MRI stuff
bem = data_dict['bem'].load()
fwd = data_dict['fwd'].load()
src = fwd['src']
trans = data_dict['trans'].load()
# load the raw data
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
# Filter for gamma
gamma=raw.copy().filter(30,50)

In [ ]:
# Let's at power - that's where we can use our Hilbert envelope, right? 
gamma_hilbert = gamma.copy()
gamma_hilbert.apply_hilbert(envelope=True)
evts, evtsid = mne.events_from_annotations(raw)
epochs_gamma_hilbert = mne.Epochs(gamma_hilbert, evts, evtsid, tmin=-0.1, tmax=0.3, preload=True)
evoked_gamma_hilbert_stim = epochs_gamma_hilbert['stim'].average()
evoked_gamma_hilbert_missingstim = epochs_gamma_hilbert['missingstim'].average()
_=evoked_gamma_hilbert_stim.plot()
_=evoked_gamma_hilbert_missingstim.plot()

In [ ]:
# Wait a second, where is it? Well, this is the difference between evoked and induced
# responses. The power in the gamma band doesn't really change that much. 
# This is actually an evoked gamma response - gamma is in phase post-stimulus, so it doesn't average out. 

In [ ]:
epochs_gamma = mne.Epochs(gamma, evts, evtsid, tmin=-0.1, tmax=0.3, preload=True)
evoked_gamma_stim = epochs_gamma['stim'].average()
#evoked_gamma_missingstim = epochs_gamma['missingstim'].average()
_=evoked_gamma_stim.plot()
#_=evoked_gamma_missingstim.plot()

In [ ]:
# Ah yes, there's our response! 

In [ ]:
# What happens if we take the Hilbert envelope of *that* 
evoked_gamma_stim_hilbert=evoked_gamma_stim.copy()
evoked_gamma_stim_hilbert.apply_hilbert(envelope=True)
_=evoked_gamma_stim_hilbert.plot()

In [ ]:
# So the trick is that if we average FIRST and THEN take the Hilbert, the in phase signals 
# add and the out of phase signals cancel out. If we take the Hilbert FIRST, before the averaging, we don't
# capture that this is really a change in phase.

In [ ]:
# How do we project that into source space? 

In [ ]:
# If there was a power change, we could do this:

# covariance for the whole time interval
cov_all_gamma = mne.compute_covariance(epochs_gamma, tmin=-0.1, tmax = 0.2, n_jobs=n_jobs)
# covariance just for the baseline
cov_baseline_gamma = mne.compute_covariance(epochs_gamma, tmin=-0.1, tmax = 0.0, n_jobs=n_jobs)
# covariance just for the post-stimulus period
cov_stim_gamma = mne.compute_covariance(epochs_gamma['stim'], tmin=0.0, tmax = 0.1, n_jobs=n_jobs)
# missing stim, cuz why not
cov_missingstim_gamma = mne.compute_covariance(epochs_gamma['missingstim'], tmin=0.0, tmax = 0.1, n_jobs=n_jobs)
# make the beamformer weights using the whole intervall
filters_gamma = mne.beamformer.make_lcmv(epochs_gamma.info, fwd, cov_all_gamma, reg=0.05, pick_ori='max-power',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')
# get a source map (not a time course) for the stim, baseline, and contrast
stc_stim_gamma = mne.beamformer.apply_lcmv_cov(cov_stim_gamma, filters_gamma)
stc_baseline_gamma = mne.beamformer.apply_lcmv_cov(cov_baseline_gamma, filters_gamma)
stc_contrast = stc_stim_gamma/stc_baseline_gamma

In [ ]:
brain=stc_contrast.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# That's not what I expected! But it's the same thing - total power isn't changing much, it's that the gamma 
# oscillations are in phase post stimulus. But how do we make an image of That?

In [ ]:
# Remember how we took the hilbert 
epochs_gamma_hilbert=epochs_gamma.copy()
# But here - we won't take the envelope - we'll keep the complex signal
epochs_gamma_hilbert.apply_hilbert()
evoked_gamma_hilbert_xform = epochs_gamma_hilbert['stim'].average()
cov_all_gamma = mne.compute_covariance(epochs_gamma_hilbert, tmin=-0.1, tmax = 0.2, n_jobs=n_jobs)
filters_gamma = mne.beamformer.make_lcmv(epochs_gamma_hilbert.info, fwd, cov_all_gamma, reg=0.05, 
                                pick_ori='max-power',
                                noise_cov = noise_cov, weight_norm='unit-noise-gain')
stc_stim_gamma = mne.beamformer.apply_lcmv(evoked_gamma_hilbert_xform, filters_gamma)
# Now here, we take the abs of the signal to get the envelope
stc_stim_gamma = abs(stc_stim_gamma)


In [ ]:
brain=stc_stim_gamma.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# placement is a little off, but there's are response! 

In [ ]:
# Let's create these images for all subjects. But this time, let's use a volumetric source space, so that 
# we can make nifti images 

In [ ]:
import nibabel as nib
# declare a few empty lists to hold the source estimates

stcs_stim=[]
stcs_missingstim=[]
stcs_contrast=[]
vol_srcs=[]
for subject in subjects:
    # filenames and load MRI files
    fs_subject = 'sub-' + subject
    data_dict = nih2mne.utilities.bids_helpers.get_mri_dict(subject,bids_root, task='airpuff')
    bem = data_dict['bem'].load()
    trans = data_dict['trans'].load()
    #Load the Raw data
    bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
    raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
    # filter for alpha        
    surface = op.join(fs_subjects_dir, fs_subject, 'bem',"inner_skull.surf")
    vol_src=mne.setup_volume_source_space(subject=fs_subject,subjects_dir=fs_subjects_dir,surface=surface)
    vol_srcs.append(vol_src)
    vol_fwd=mne.make_forward_solution(raw.info,trans,vol_src,bem=bem)
    gamma=raw.copy().filter(30,50)

    epochs_gamma = mne.Epochs(gamma, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)
    epochs_gamma_hilbert=epochs_gamma.copy()
    epochs_gamma_hilbert.apply_hilbert()

    evoked_gamma_hilbert_stim = epochs_gamma_hilbert['stim'].average()
    evoked_gamma_hilbert_missingstim = epochs_gamma_hilbert['missingstim'].average()
    
    cov_all_gamma = mne.compute_covariance(epochs_gamma_hilbert, tmin=-0.1, tmax = 0.2, n_jobs=n_jobs)
    filters_gamma = mne.beamformer.make_lcmv(epochs_gamma_hilbert.info, vol_fwd, cov_all_gamma, reg=0.05, 
                                pick_ori='max-power',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')
    stc_stim_gamma = mne.beamformer.apply_lcmv(evoked_gamma_hilbert_stim, filters_gamma)
    stc_stim_gamma = abs(stc_stim_gamma)
    stc_missingstim_gamma = mne.beamformer.apply_lcmv(evoked_gamma_hilbert_missingstim, filters_gamma)
    stc_missingstim_gamma = abs(stc_missingstim_gamma)
    stcs_stim.append(stc_stim_gamma)
    stcs_missingstim.append(stc_missingstim_gamma)
    #niftiimg=stc_stim_gamma.as_volume(vol_src,dest='mri')
    #fname=op.join(output_dir,f'{fs_subject}_stim_image.nii')
    #nib.save(niftiimg,fname)
    #niftiimg=stc_missingstim_gamma.as_volume(vol_src,dest='mri')
    #fname=op.join(output_dir,f'{fs_subject}_missingstim_image.nii')
    #nib.save(niftiimg,fname)

In [ ]:
# Look at any of the subjects
subjnum=0
_=stcs_stim[subjnum].plot(mode="stat_map", src=vol_srcs[subjnum], subject=fs_subjects[subjnum], 
                          subjects_dir=fs_subjects_dir)
_=stcs_missingstim[subjnum].plot(mode="stat_map", src=vol_srcs[subjnum], subject=fs_subjects[subjnum], 
                          subjects_dir=fs_subjects_dir)